In [ ]:
pip install wehoop-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 350.2/350.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 2.3.0
    Uninstalling PyJWT-2.3.0:
      Successfully uninstalled PyJWT-2.3.0


In [ ]:
import wehoop
import pandas as pd
from datetime import datetime

In [ ]:
functions = [func for func in dir(wehoop) if callable(getattr(wehoop, func))]

print("Functions in the wehoop package:")
for func in functions:
    print(func)


Functions in the wehoop package:
Callable
Iterator
List
Optional
SeasonNotFoundError
Union
download
load_wbb_pbp
load_wbb_player_boxscore
load_wbb_schedule
load_wbb_team_boxscore
load_wnba_pbp
load_wnba_player_boxscore
load_wnba_schedule
load_wnba_team_boxscore
wbb_calendar
wnba_calendar


In [ ]:
def get_schedule(season):
    schedule = wehoop.load_wbb_schedule([season])
    schedule_df = pd.DataFrame(schedule)

    schedule_df.to_csv('wbb_schedule.csv', index=False)

    return schedule_df

In [ ]:
def get_boxscores():
  boxscores = wehoop.load_wbb_team_boxscore([season])
  # boxscore_df.to_csv('test.csv', index=False)

  pd.set_option('display.max_columns', None)

  cols_to_drop = [
    'season', 'season_type', 'game_date_time', 'team_uid', 'team_slug', 'team_location',
    'team_name', 'team_abbreviation', 'team_short_display_name', 'team_color', 'team_alternate_color', 'team_logo',
    'opponent_team_id', 'opponent_team_uid', 'opponent_team_slug', 'opponent_team_location', 'opponent_team_name',
    'opponent_team_abbreviation', 'opponent_team_display_name', 'opponent_team_short_display_name',
    'opponent_team_color', 'opponent_team_alternate_color', 'opponent_team_logo', 'opponent_team_score'
  ]

  boxscores = boxscores.drop(columns=cols_to_drop)
  boxscores = boxscores.merge(boxscores, on=['game_date', 'game_id'], suffixes=('', '_opponent'))
  boxscores = boxscores[boxscores['team_id'] != boxscores['team_id_opponent']]
  boxscores
  return boxscores

In [ ]:
def get_future_opps(team, date, season):
    schedule_df = get_schedule(season)
    future_games = schedule_df[schedule_df["game_date"] > date]

    team_games = future_games[(future_games["home_display_name"] == team) | (future_games["away_display_name"] == team)]

    opp_dict = {team: []}

    for index, game in team_games.iterrows():
        if game["home_display_name"] == team:
            opponent = game["away_display_name"]
        else:
            opponent = game["home_display_name"]
        opp_dict[team].append({"date": game["game_date"], "opponent": opponent})

    return opp_dict

In [ ]:
def get_past_opps(boxscores, futureOpp, date):
    date = pd.to_datetime(date)

    past_opps = boxscores[(boxscores['team_display_name'] == futureOpp) & (boxscores['game_date'] < date)]

    return past_opps

In [ ]:
team = "Clemson Tigers"
season = "2022"
date = "2/1/2022"
date = datetime.strptime(date, "%m/%d/%Y").date()

In [ ]:
schedule_df = get_schedule(season)
schedule_df

/usr/local/lib/python3.10/dist-packages/wehoop/wbb/__init__.py:118: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(i_data)


,id,uid,date,attendance,time_valid,neutral_site,conference_competition,recent,start_date,notes_type,...,groups_id,groups_name,groups_short_name,groups_is_conference,tournament_id,game_date_time,game_date,PBP,team_box,player_box
0,401414529,s:40~l:54~e:401414529~c:401414529,2022-04-04T00:00Z,18304,True,True,False,False,2022-04-04T00:00Z,event,...,NaN,None,None,None,22.0,2022-04-03 20:00:00-04:00,2022-04-03,True,True,True
1,401415936,s:40~l:54~e:401415936~c:401415936,2022-04-02T19:00Z,5263,True,True,False,False,2022-04-02T19:00Z,event,...,NaN,None,None,None,37.0,2022-04-02 15:00:00-04:00,2022-04-02,True,True,True
2,401414527,s:40~l:54~e:401414527~c:401414527,2022-04-02T01:30Z,18268,True,True,False,False,2022-04-02T01:30Z,event,...,NaN,None,None,None,22.0,2022-04-01 21:30:00-04:00,2022-04-01,True,True,True
3,401414528,s:40~l:54~e:401414528~c:401414528,2022-04-01T23:00Z,0,True,True,False,False,2022-04-01T23:00Z,event,...,NaN,None,None,None,22.0,2022-04-01 19:00:00-04:00,2022-04-01,True,True,True
4,401415938,s:40~l:54~e:401415938~c:401415938,2022-04-01T00:00Z,5227,True,False,False,False,2022-04-01T00:00Z,event,...,NaN,None,None,None,37.0,2022-03-31 20:00:00-04:00,2022-03-31,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6003,401375952,s:40~l:54~e:401375952~c:401375952,2021-11-09T16:30Z,100,True,False,False,False,2021-11-09T16:30Z,,...,NaN,None,None,None,NaN,2021-11-09 11:30:00-05:00,2021-11-09,True,True,True
6004,401375951,s:40~l:54~e:401375951~c:401375951,2021-11-09T16:30Z,506,True,False,False,False,2021-11-09T16:30Z,,...,NaN,None,None,None,NaN,2021-11-09 11:30:00-05:00,2021-11-09,True,True,True
6005,401380643,s:40~l:54~e:401380643~c:401380643,2021-11-09T16:00Z,4613,True,False,False,False,2021-11-09T16:00Z,,...,NaN,None,None,None,NaN,2021-11-09 11:00:00-05:00,2021-11-09,True,True,True
6006,401376678,s:40~l:54~e:401376678~c:401376678,2021-11-09T16:00Z,1155,True,False,False,False,2021-11-09T16:00Z,,...,NaN,None,None,None,NaN,2021-11-09 11:00:00-05:00,2021-11-09,True,True,True


In [ ]:
boxscores = get_boxscores()
boxscores

/usr/local/lib/python3.10/dist-packages/wehoop/wbb/__init__.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(i_data)


,game_id,game_date,team_id,team_display_name,team_home_away,team_score,team_winner,assists,blocks,defensive_rebounds,field_goal_pct,field_goals_made,field_goals_attempted,flagrant_fouls,fouls,free_throw_pct,free_throws_made,free_throws_attempted,largest_lead,offensive_rebounds,steals,team_turnovers,technical_fouls,three_point_field_goal_pct,three_point_field_goals_made,three_point_field_goals_attempted,total_rebounds,total_technical_fouls,total_turnovers,turnovers,team_id_opponent,team_display_name_opponent,team_home_away_opponent,team_score_opponent,team_winner_opponent,assists_opponent,blocks_opponent,defensive_rebounds_opponent,field_goal_pct_opponent,field_goals_made_opponent,field_goals_attempted_opponent,flagrant_fouls_opponent,fouls_opponent,free_throw_pct_opponent,free_throws_made_opponent,free_throws_attempted_opponent,largest_lead_opponent,offensive_rebounds_opponent,steals_opponent,team_turnovers_opponent,technical_fouls_opponent,three_point_field_goal_pct_opponent,three_point_field_goals_made_opponent,three_point_field_goals_attempted_opponent,total_rebounds_opponent,total_technical_fouls_opponent,total_turnovers_opponent,turnovers_opponent
1,401414529,2022-04-03,41,UConn Huskies,away,49,False,14,5,18,40.7,22,54,0,21,25.0,1,4,0,6,4,1,0,25.0,4,16,24,0,15,15,2579,South Carolina Gamecocks,home,64,True,9,4,28,36.7,22,60,0,11,65.4,17,26,18,21,6,0,0,18.8,3,16,49,0,14,14
2,401414529,2022-04-03,2579,South Carolina Gamecocks,home,64,True,9,4,28,36.7,22,60,0,11,65.4,17,26,18,21,6,0,0,18.8,3,16,49,0,14,14,41,UConn Huskies,away,49,False,14,5,18,40.7,22,54,0,21,25.0,1,4,0,6,4,1,0,25.0,4,16,24,0,15,15
5,401415936,2022-04-02,2550,Seton Hall Pirates,away,50,False,7,3,23,26.6,17,64,0,19,80.0,12,15,0,9,10,0,1,22.2,4,18,32,1,11,11,2571,South Dakota State Jackrabbits,home,82,True,20,3,40,44.8,30,67,0,14,66.7,14,21,37,17,3,1,0,34.8,8,23,57,0,15,15
6,401415936,2022-04-02,2571,South Dakota State Jackrabbits,home,82,True,20,3,40,44.8,30,67,0,14,66.7,14,21,37,17,3,1,0,34.8,8,23,57,0,15,15,2550,Seton Hall Pirates,away,50,False,7,3,23,26.6,17,64,0,19,80.0,12,15,0,9,10,0,1,22.2,4,18,32,1,11,11
9,401414527,2022-04-01,41,UConn Huskies,away,63,True,14,2,34,36.8,21,57,0,16,80.0,16,20,8,12,5,0,0,35.7,5,14,46,0,19,19,24,Stanford Cardinal,home,58,False,10,3,25,34.8,23,66,0,16,61.5,8,13,1,12,11,0,0,17.4,4,23,37,0,11,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22038,401380643,2021-11-09,120,Maryland Terrapins,home,97,True,16,4,28,45.7,37,81,0,13,82.4,14,17,30,21,13,0,0,47.4,9,19,49,0,11,11,2344,Longwood Lancers,away,67,False,13,7,24,39.0,23,59,0,15,77.8,14,18,4,9,4,0,0,31.8,7,22,33,0,19,19
22041,401376678,2021-11-09,2010,Alabama A&M Bulldogs,away,52,False,11,10,25,30.3,20,66,0,17,76.9,10,13,4,15,5,1,0,16.7,2,12,40,0,12,12,2168,Dayton Flyers,home,73,True,10,5,32,43.5,27,62,0,17,68.8,11,16,21,12,3,0,0,53.3,8,15,44,0,11,11
22042,401376678,2021-11-09,2168,Dayton Flyers,home,73,True,10,5,32,43.5,27,62,0,17,68.8,11,16,21,12,3,0,0,53.3,8,15,44,0,11,11,2010,Alabama A&M Bulldogs,away,52,False,11,10,25,30.3,20,66,0,17,76.9,10,13,4,15,5,1,0,16.7,2,12,40,0,12,12
22045,401375181,2021-11-09,2427,UNC Asheville Bulldogs,away,48,False,10,2,29,29.9,20,67,0,12,100.0,2,2,6,12,8,1,0,24.0,6,25,41,0,13,13,2430,UNC Greensboro Spartans,home,52,True,8,8,35,33.9,21,62,0,13,70.0,7,10,16,13,7,0,0,18.8,3,16,48,0,15,15


In [ ]:
future_opps = get_future_opps(team, date, season)
for team, games in future_opps.items():
    print(f"Team: {team}")
    for game in games:
        print(f"Date: {game['date']}, Opponent: {game['opponent']}")

/usr/local/lib/python3.10/dist-packages/wehoop/wbb/__init__.py:118: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(i_data)


Team: Clemson Tigers
Date: 2022-03-03, Opponent: Virginia Tech Hokies
Date: 2022-03-02, Opponent: Syracuse Orange
Date: 2022-02-27, Opponent: Miami Hurricanes
Date: 2022-02-24, Opponent: Notre Dame Fighting Irish
Date: 2022-02-20, Opponent: Duke Blue Devils
Date: 2022-02-13, Opponent: Georgia Tech Yellow Jackets
Date: 2022-02-10, Opponent: Wake Forest Demon Deacons
Date: 2022-02-08, Opponent: Virginia Tech Hokies
Date: 2022-02-06, Opponent: Virginia Cavaliers
Date: 2022-02-03, Opponent: Louisville Cardinals


In [ ]:
futureOpp = future_opps[list(future_opps.keys())[0]][0]['opponent']
futureOpp

'Virginia Tech Hokies'

In [ ]:
get_past_opps(boxscores, futureOpp, date)

<ipython-input-7-fa57e1a9a5da>:4: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  past_opps = boxscores[(boxscores['team_display_name'] == futureOpp) & (boxscores['game_date'] < date)]


,game_id,game_date,team_id,team_display_name,team_home_away,team_score,team_winner,assists,blocks,defensive_rebounds,field_goal_pct,field_goals_made,field_goals_attempted,flagrant_fouls,fouls,free_throw_pct,free_throws_made,free_throws_attempted,largest_lead,offensive_rebounds,steals,team_turnovers,technical_fouls,three_point_field_goal_pct,three_point_field_goals_made,three_point_field_goals_attempted,total_rebounds,total_technical_fouls,total_turnovers,turnovers,team_id_opponent,team_display_name_opponent,team_home_away_opponent,team_score_opponent,team_winner_opponent,assists_opponent,blocks_opponent,defensive_rebounds_opponent,field_goal_pct_opponent,field_goals_made_opponent,field_goals_attempted_opponent,flagrant_fouls_opponent,fouls_opponent,free_throw_pct_opponent,free_throws_made_opponent,free_throws_attempted_opponent,largest_lead_opponent,offensive_rebounds_opponent,steals_opponent,team_turnovers_opponent,technical_fouls_opponent,three_point_field_goal_pct_opponent,three_point_field_goals_made_opponent,three_point_field_goals_attempted_opponent,total_rebounds_opponent,total_technical_fouls_opponent,total_turnovers_opponent,turnovers_opponent
9185,401369559,2022-01-27,259,Virginia Tech Hokies,away,71,True,10,4,24,43.9,25,57,0,11,58.8,10,17,29,14,6,0,0,39.3,11,28,38,0,10,10,258,Virginia Cavaliers,home,42,False,6,4,21,34.7,17,49,0,20,66.7,4,6,3,10,2,0,0,44.4,4,9,31,0,19,19
9677,401369548,2022-01-23,259,Virginia Tech Hokies,away,45,False,8,3,32,29.3,17,58,0,9,100.0,4,4,6,6,3,1,0,30.4,7,23,38,0,13,13,152,NC State Wolfpack,home,51,True,7,6,35,32.4,22,68,0,13,75.0,3,4,15,14,4,0,0,26.7,4,15,49,0,11,11
10522,401369539,2022-01-20,259,Virginia Tech Hokies,home,75,True,17,1,27,40.4,23,57,0,14,88.0,22,25,21,11,1,1,0,36.8,7,19,38,0,12,12,221,Pittsburgh Panthers,away,65,False,10,7,23,36.2,25,69,0,24,60.0,9,15,2,17,3,1,0,42.9,6,14,40,0,11,11
10930,401406290,2022-01-18,259,Virginia Tech Hokies,home,69,True,17,7,23,38.3,23,60,0,15,94.1,16,17,18,13,4,0,0,31.8,7,22,36,0,9,9,258,Virginia Cavaliers,away,52,False,11,4,24,36.0,18,50,0,21,76.5,13,17,3,10,6,1,0,37.5,3,8,34,0,16,16
11989,401369525,2022-01-13,259,Virginia Tech Hokies,away,65,True,10,3,22,36.9,24,65,0,16,62.5,10,16,12,17,4,1,0,33.3,7,21,39,0,9,9,150,Duke Blue Devils,home,54,False,11,3,25,35.7,20,56,0,20,91.7,11,12,7,14,6,0,0,27.3,3,11,39,0,14,14
12297,401369521,2022-01-09,259,Virginia Tech Hokies,away,46,False,9,2,28,31.0,18,58,0,18,75.0,6,8,2,11,5,1,0,16.7,4,24,39,0,21,21,153,North Carolina Tar Heels,home,71,True,14,3,28,43.8,28,64,0,17,63.6,7,11,32,9,9,0,0,44.4,8,18,37,0,12,12
13189,401369506,2022-01-02,259,Virginia Tech Hokies,away,66,True,7,6,30,40.7,22,54,0,12,83.3,15,18,16,6,7,0,0,30.4,7,23,36,0,13,13,154,Wake Forest Demon Deacons,home,53,False,8,1,24,33.3,20,60,0,19,69.2,9,13,0,11,7,0,0,21.1,4,19,35,0,15,15
13590,401369498,2021-12-30,259,Virginia Tech Hokies,home,77,True,15,3,32,43.6,24,55,0,16,100.0,20,20,31,6,3,0,0,40.9,9,22,38,0,9,9,150,Duke Blue Devils,away,55,False,9,3,25,30.4,21,69,0,14,66.7,10,15,2,17,0,0,0,16.7,3,18,42,0,12,12
14513,401369479,2021-12-19,259,Virginia Tech Hokies,away,92,True,20,4,25,56.4,31,55,0,17,80.0,20,25,22,8,4,0,0,47.6,10,21,33,0,9,9,52,Florida State Seminoles,home,75,False,15,4,18,46.7,28,60,0,21,80.0,12,15,5,6,4,0,0,41.2,7,17,24,0,9,9
15470,401369468,2021-12-12,259,Virginia Tech Hokies,home,89,True,17,4,28,52.9,36,68,0,13,64.3,9,14,49,11,6,1,0,28.6,8,28,39,0,6,6,2515,Radford Highlanders,away,40,False,6,2,22,25.5,13,51,0,17,75.0,9,12,0,9,3,1,0,55.6,5,9,31,0,22,22
